# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import datetime
%matplotlib inline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

If cvFitLine() returns normalized vector (vx,vy) and point (x0,y0), then the equation of the line is

(x,y) = (x0,y0) + t*(vx,vy)

where t runs from −∞ to +∞.

If we need to draw a big line instead of solving for the boundaries following is an example. eg:

cv.Line(img, (x0-m*vx[0], y0-m*vy[0]), (x0+m*vx[0], y0+m*vy[0]), (0,0,0))

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [2]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    #return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[0, 0, 255], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    initial_hough = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    # Keep only lines with slope between 25 deg and 70 deg
    filtered_lines = []
    Min_angle = math.tan(math.pi*25/180) # 25 degrees
    Max_angle = math.tan(math.pi*70/180)  # 70 degrees
    for line in lines:
        for x1,y1,x2,y2 in line:
            if x1 !=x2:
                slope= ((y2-y1)/(x2-x1))
                if abs(slope) > Min_angle and abs(slope) < Max_angle:
                    filtered_lines.append([x1,y1,x2,y2,slope])
                    
                
    
    imshape = img.shape
    # Sort the lines by their slope values.
    lines = sorted(filtered_lines,key=lambda coordinates: coordinates[4])
    
    Left_Lines=[]
    Right_Lines=[]
    All_left=[]
    All_right=[]
    
    # Separate left lane lines and right lane lines based on value of slope and x coordinate
    for line in lines:
        #print (line)
        x1,y1,x2,y2,slope = line
        if slope <0 and x1<float(imshape[1]/2) and x2 <float(imshape[1]/2):
            # Left Line
            color=[0, 0, 255]
            Left_Lines.append((x1,y1))
            Left_Lines.append((x2,y2))
            All_left.append([x1,y1,x2,y2])
            
        elif slope >0 and x1>float(imshape[1]/2) and x2 >float(imshape[1]/2):
            # Right Line
            color=[255, 0, 0]
            Right_Lines.append((x1,y1))
            Right_Lines.append((x2,y2))
            All_right.append([x1,y1,x2,y2])

    print ("LeftLanesLength",len(Left_Lines))
    print ("RightLanesLength",len(Right_Lines))
    
    
    param = 0
    reps = 0.01 
    aeps = 0.01
    distL12 = 4
    
    y_max = int(imshape[0])
    print ("y_max ",y_max)
    y_min = int((imshape[0]*2.0/3.0))
    print ("y_min ",y_min)
    
    # Fit a line through all points in left lane and do similarly for right lane.
    # Extend the fitted line to the boundaries of the mask.
    if Left_Lines:
        vx_left, vy_left, cx_left, cy_left  = cv2.fitLine(np.array(Left_Lines), distL12, param, reps, aeps) 
        print ("vx_left, vy_left, cx_left, cy_left ", vx_left, vy_left, cx_left, cy_left)
        m_left_bottom = float(y_max - cy_left)/float(vy_left)  
        m_left_up = float(y_min - cy_left)/float(vy_left)
        point_x1_left = int(cx_left+m_left_bottom*vx_left)
        point_y1_left = int(cy_left+m_left_bottom*vy_left)
        point_x2_left = int(cx_left+m_left_up*vx_left)
        point_y2_left = int(cy_left+m_left_up*vy_left)
        cv2.line(img, (point_x1_left, point_y1_left), (point_x2_left, point_y2_left), (255,0,0),3)
        cv2.line(initial_hough, (point_x1_left, point_y1_left), (point_x2_left, point_y2_left), (255,255,0),1)
    if Right_Lines:
        vx_right, vy_right, cx_right, cy_right  = cv2.fitLine(np.array(Right_Lines), distL12, param, reps, aeps)
        print ("vx_right , vy_right , cx_right , cy_right  ", vx_right , vy_right , cx_right , cy_right )
        m_right_bottom = float(y_max - cy_right)/float(vy_right)  
        m_right_up = float(y_min - cy_right)/float(vy_right)
        point_x1_right = int(cx_right+m_right_bottom*vx_right)
        point_y1_right = int(cy_right+m_right_bottom*vy_right)
        point_x2_right = int(cx_right+m_right_up*vx_right)
        point_y2_right = int(cy_right+m_right_up*vy_right)
        cv2.line(img, (point_x1_right, point_y1_right), (point_x2_right, point_y2_right), (0,0,255),3)
        cv2.line(initial_hough, (point_x1_right, point_y1_right), (point_x2_right, point_y2_right), (0,255,255),1)
            

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [3]:
import os
os.listdir("test_images/")

['solidWhiteRight.jpg',
 'solidWhiteCurve.jpg',
 'solidYellowCurve.jpg',
 'solidYellowLeft.jpg',
 'whiteCarLaneSwitch.jpg',
 'solidYellowCurve2.jpg']

run your solution on all test_images and make copies into the test_images directory).

In [4]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.

for imageFile in os.listdir("test_images/"):
    #reading in an image
    imageName = os.path.join("test_images",imageFile)
    image = cv2.imread(imageName)
    #printing out some stats and plotting
    print('This image is:', imageName, 'with dimesions:', image.shape)
    # Convert image to grayscale
    gray = grayscale(image)


    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 5
    blur_gray = gaussian_blur(gray,5)

    # Define our parameters for Canny and apply
    low_threshold = 50
    high_threshold = 150
    edges = canny(blur_gray, low_threshold, high_threshold)

    # Define Region of Interest
    imshape = edges.shape
    offset_x = imshape[1]/10
    offset_y = imshape[0]/8
    vertices = np.array([[(0,imshape[0]),(imshape[1]/2-offset_x, imshape[0]/2+offset_y), (imshape[1]/2+offset_x, imshape[0]/2+offset_y), (imshape[1],imshape[0])]], dtype=np.int32)
    ROI = region_of_interest(edges,vertices)
    # cv2.imwrite("output_ROI.jpg",ROI) 

    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 40 #minimum number of pixels making up a line
    max_line_gap = 20    # maximum gap in pixels between connectable line segments

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    line_image = hough_lines(ROI, rho, theta, threshold, min_line_len, max_line_gap)

    final_image =  weighted_img(line_image,image,α=0.4,β=6.0)

    #lanesFilename = os.path.join('output',imageFile.split('.')[0]+'_outputLanes.jpg')
    finalFilename = os.path.join('test_images',imageFile.split('.')[0]+'_outputFinal.jpg')
    #cv2.imwrite(lanesFilename,line_image)
    cv2.imwrite(finalFilename,final_image)

This image is: test_images/solidWhiteRight.jpg with dimesions: (540, 960, 3)
LeftLanesLength 14
RightLanesLength 14
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.81145144] [-0.58441985] [ 382.33453369] [ 376.96484375]
vx_right , vy_right , cx_right , cy_right   [ 0.83492512] [ 0.55036354] [ 699.97125244] [ 449.92160034]
This image is: test_images/solidWhiteCurve.jpg with dimesions: (540, 960, 3)
LeftLanesLength 12
RightLanesLength 12
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.7883594] [-0.615215] [ 395.19445801] [ 375.23718262]
vx_right , vy_right , cx_right , cy_right   [ 0.86496711] [ 0.50182861] [ 685.79321289] [ 425.37841797]
This image is: test_images/solidYellowCurve.jpg with dimesions: (540, 960, 3)
LeftLanesLength 18
RightLanesLength 12
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.81129473] [-0.58463734] [ 346.81228638] [ 405.58526611]
vx_right , vy_right , cx_right , cy_right   [ 0.86029822] [ 0.50979114] [ 594.04241943] [ 38

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [5]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [6]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    # Convert image to grayscale
    gray = grayscale(image)

    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 5
    blur_gray = gaussian_blur(gray,5)

    # Define our parameters for Canny and apply
    low_threshold = 50
    high_threshold = 150
    edges = canny(blur_gray, low_threshold, high_threshold)

    # Define Region of Interest
    imshape = edges.shape
    offset_x = imshape[1]/10
    offset_y = imshape[0]/8
    vertices = np.array([[(imshape[1]/16,imshape[0]),(imshape[1]/2-offset_x, int(imshape[0]*2.0/3.0)), (imshape[1]/2+offset_x, int(imshape[0]*2.0/3.0)), (imshape[1]-(imshape[1]/16),imshape[0])]], dtype=np.int32)
    ROI = region_of_interest(edges,vertices)

    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 10     # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 20 #minimum number of pixels making up a line
    max_line_gap = 40    # maximum gap in pixels between connectable line segments

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    line_image = hough_lines(ROI, rho, theta, threshold, min_line_len, max_line_gap)

    final_image =  weighted_img(line_image,image,α=0.4,β=6.0)

    return final_image

Let's try the one with the solid white lane on the right first ...

In [7]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

LeftLanesLength 24
RightLanesLength 28
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.79357409] [-0.60847366] [ 326.35855103] [ 416.98703003]
vx_right , vy_right , cx_right , cy_right   [ 0.85651451] [ 0.516123] [ 681.65643311] [ 426.71673584]
[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


  2%|▏         | 5/222 [00:00<00:04, 46.31it/s]

LeftLanesLength 24
RightLanesLength 28
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.79357409] [-0.60847366] [ 326.35855103] [ 416.98703003]
vx_right , vy_right , cx_right , cy_right   [ 0.85651451] [ 0.516123] [ 681.65643311] [ 426.71673584]
LeftLanesLength 24
RightLanesLength 22
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.80991185] [-0.58655161] [ 325.19848633] [ 415.95007324]
vx_right , vy_right , cx_right , cy_right   [ 0.835899] [ 0.54888332] [ 706.22277832] [ 445.84002686]
LeftLanesLength 22
RightLanesLength 20
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.79192424] [-0.61061937] [ 291.37817383] [ 444.14312744]
vx_right , vy_right , cx_right , cy_right   [ 0.84393299] [ 0.5364486] [ 730.39581299] [ 463.49981689]
LeftLanesLength 20
RightLanesLength 20
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.80644369] [-0.59131092] [ 306.98632812] [ 430.33337402]
vx_right , vy_right , cx_right , cy_right   [ 0.84138191] [ 0.540

  8%|▊         | 17/222 [00:00<00:04, 49.99it/s]

LeftLanesLength 20
RightLanesLength 24
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.82372868] [-0.56698418] [ 335.90042114] [ 410.39358521]
vx_right , vy_right , cx_right , cy_right   [ 0.84971881] [ 0.5272361] [ 666.23846436] [ 425.99111938]
LeftLanesLength 24
RightLanesLength 24
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.79793173] [-0.60274786] [ 315.62136841] [ 425.73858643]
vx_right , vy_right , cx_right , cy_right   [ 0.85292882] [ 0.52202719] [ 650.13464355] [ 415.59643555]
LeftLanesLength 28
RightLanesLength 24
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.82282358] [-0.56829691] [ 287.61965942] [ 439.74487305]
vx_right , vy_right , cx_right , cy_right   [ 0.85150045] [ 0.52435386] [ 727.44055176] [ 460.7487793]
LeftLanesLength 32
RightLanesLength 16
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.82165593] [-0.56998378] [ 248.38789368] [ 465.2543335]
vx_right , vy_right , cx_right , cy_right   [ 0.8555643] [ 0.51

...

 97%|█████████▋| 215/222 [00:07<00:00, 25.14it/s]

LeftLanesLength 16
RightLanesLength 8
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.79574788] [-0.60562801] [ 374.52487183] [ 388.01696777]
vx_right , vy_right , cx_right , cy_right   [ 0.86496884] [ 0.50182557] [ 748.98986816] [ 462.26306152]
LeftLanesLength 12
RightLanesLength 10
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.7780593] [-0.62819082] [ 373.37130737] [ 388.77642822]
vx_right , vy_right , cx_right , cy_right   [ 0.87319005] [ 0.48737988] [ 725.30023193] [ 443.77163696]
LeftLanesLength 24
RightLanesLength 6
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.79906797] [-0.60124069] [ 358.91699219] [ 401.87692261]
vx_right , vy_right , cx_right , cy_right   [ 0.86617172] [ 0.49974653] [ 708.36871338] [ 439.36962891]
LeftLanesLength 14
RightLanesLength 6
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.80118704] [-0.598414] [ 359.25979614] [ 401.56576538]
vx_right , vy_right , cx_right , cy_right   [ 0.86697304] [ 0.4983

100%|█████████▉| 221/222 [00:07<00:00, 28.71it/s]

LeftLanesLength 28
RightLanesLength 22
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.78017986] [-0.62555528] [ 329.72232056] [ 425.89916992]
vx_right , vy_right , cx_right , cy_right   [ 0.87143064] [ 0.49051881] [ 720.80181885] [ 441.93878174]
LeftLanesLength 28
RightLanesLength 12
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.79622424] [-0.60500163] [ 316.97125244] [ 433.27505493]
vx_right , vy_right , cx_right , cy_right   [ 0.86394471] [ 0.50358665] [ 674.41766357] [ 421.13574219]
LeftLanesLength 22
RightLanesLength 14
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.77311367] [-0.63426751] [ 314.70202637] [ 437.50653076]
vx_right , vy_right , cx_right , cy_right   [ 0.87649024] [ 0.48141965] [ 705.47253418] [ 433.48468018]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 49.7 s, sys: 1.94 s, total: 51.7 s
Wall time: 8.2 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [9]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

LeftLanesLength 26
RightLanesLength 18
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.84258944] [-0.53855646] [ 266.5770874] [ 447.35931396]
vx_right , vy_right , cx_right , cy_right   [ 0.83192778] [ 0.55488396] [ 672.12872314] [ 429.57037354]
[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


  1%|          | 5/682 [00:00<00:14, 48.28it/s]

LeftLanesLength 26
RightLanesLength 18
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.84258944] [-0.53855646] [ 266.5770874] [ 447.35931396]
vx_right , vy_right , cx_right , cy_right   [ 0.83192778] [ 0.55488396] [ 672.12872314] [ 429.57037354]
LeftLanesLength 20
RightLanesLength 26
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.84677356] [-0.53195351] [ 272.06286621] [ 445.03762817]
vx_right , vy_right , cx_right , cy_right   [ 0.83323234] [ 0.55292296] [ 670.54290771] [ 429.75976562]
LeftLanesLength 24
RightLanesLength 34
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.84430367] [-0.53586501] [ 278.91757202] [ 441.03912354]
vx_right , vy_right , cx_right , cy_right   [ 0.818138] [ 0.57502186] [ 696.80474854] [ 451.21606445]
LeftLanesLength 20
RightLanesLength 16
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.85578579] [-0.51733035] [ 274.92437744] [ 439.53662109]
vx_right , vy_right , cx_right , cy_right   [ 0.82635671] [ 0.5

  2%|▏         | 11/682 [00:00<00:13, 50.99it/s]

LeftLanesLength 22
RightLanesLength 16
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.83899283] [-0.54414248] [ 350.58163452] [ 398.0211792]
vx_right , vy_right , cx_right , cy_right   [ 0.84147924] [ 0.5402894] [ 627.56903076] [ 396.08459473]
LeftLanesLength 18
RightLanesLength 16
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.84101379] [-0.54101366] [ 281.14749146] [ 438.77328491]
vx_right , vy_right , cx_right , cy_right   [ 0.83320564] [ 0.55296326] [ 637.81738281] [ 405.32223511]


...

 99%|█████████▉| 676/682 [00:25<00:00, 26.59it/s]

LeftLanesLength 22
RightLanesLength 16
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.80579799] [-0.5921905] [ 304.24057007] [ 436.02682495]
vx_right , vy_right , cx_right , cy_right   [ 0.84873646] [ 0.52881604] [ 688.4107666] [ 434.8855896]
LeftLanesLength 22
RightLanesLength 24
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.80198175] [-0.59734851] [ 320.11706543] [ 427.24069214]
vx_right , vy_right , cx_right , cy_right   [ 0.86486495] [ 0.50200456] [ 723.753479] [ 452.59689331]
LeftLanesLength 18
RightLanesLength 26
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.80124211] [-0.59834027] [ 296.83822632] [ 444.24441528]
vx_right , vy_right , cx_right , cy_right   [ 0.86193705] [ 0.50701529] [ 679.15942383] [ 427.18878174]
LeftLanesLength 10
RightLanesLength 24
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.80191737] [-0.597435] [ 312.3067627] [ 434.03430176]
vx_right , vy_right , cx_right , cy_right   [ 0.85632318] [ 0.516440

100%|█████████▉| 681/682 [00:26<00:00, 25.97it/s]

LeftLanesLength 10
RightLanesLength 12
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.81544578] [-0.57883346] [ 334.09365845] [ 414.75964355]
vx_right , vy_right , cx_right , cy_right   [ 0.85969055] [ 0.51081526] [ 603.79199219] [ 384.26074219]
LeftLanesLength 20
RightLanesLength 18
y_max  540
y_min  360
vx_left, vy_left, cx_left, cy_left  [ 0.81620878] [-0.57775712] [ 278.0151062] [ 453.39920044]
vx_right , vy_right , cx_right , cy_right   [ 0.84840101] [ 0.52935404] [ 654.25152588] [ 414.8666687]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 2min 50s, sys: 6 s, total: 2min 56s
Wall time: 26.8 s


In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!

## Reflections Answer

Shortcomings of algorithm:
1) Doesn't perform well in shadows and dark conditions.
2) Lines are not very Smooth between frames in challenge video.

Improvements that could be done:
1) Some kind of temporal filtering on slope can help avoid jitter between frames.
2) Exploring any other color space like HSV etc and extracting lines from other color spaces could improve the detection of lines in shadows and dark conditions. 
3) Also adaptive thresholding algorithms can be tried to make it more robust. Other edge detectors like Sobel can improve the performance.


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [11]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

LeftLanesLength 36
RightLanesLength 20
y_max  720
y_min  480
vx_left, vy_left, cx_left, cy_left  [ 0.82395411] [-0.56665653] [ 500.7059021] [ 524.6239624]
vx_right , vy_right , cx_right , cy_right   [ 0.85144174] [ 0.52444923] [ 903.19903564] [ 570.34307861]
[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


  1%|          | 3/251 [00:00<00:08, 27.99it/s]

LeftLanesLength 36
RightLanesLength 20
y_max  720
y_min  480
vx_left, vy_left, cx_left, cy_left  [ 0.82395411] [-0.56665653] [ 500.7059021] [ 524.6239624]
vx_right , vy_right , cx_right , cy_right   [ 0.85144174] [ 0.52444923] [ 903.19903564] [ 570.34307861]
LeftLanesLength 34
RightLanesLength 18
y_max  720
y_min  480
vx_left, vy_left, cx_left, cy_left  [ 0.83422935] [-0.55141765] [ 472.79064941] [ 540.73754883]
vx_right , vy_right , cx_right , cy_right   [ 0.84966618] [ 0.52732092] [ 881.402771] [ 555.40093994]
LeftLanesLength 46
RightLanesLength 14
y_max  720
y_min  480
vx_left, vy_left, cx_left, cy_left  [ 0.88992137] [-0.45611396] [ 391.79989624] [ 578.58843994]
vx_right , vy_right , cx_right , cy_right   [ 0.86498845] [ 0.50179178] [ 895.48638916] [ 560.38085938]
LeftLanesLength 28
RightLanesLength 12
y_max  720
y_min  480
vx_left, vy_left, cx_left, cy_left  [ 0.83289498] [-0.55343109] [ 423.03497314] [ 575.26708984]
vx_right , vy_right , cx_right , cy_right   [ 0.85888112] [ 0.51

  4%|▎         | 9/251 [00:00<00:08, 28.49it/s]

LeftLanesLength 48
RightLanesLength 14
y_max  720
y_min  480
vx_left, vy_left, cx_left, cy_left  [ 0.80102783] [-0.59862709] [ 473.39929199] [ 550.14465332]
vx_right , vy_right , cx_right , cy_right   [ 0.85242981] [ 0.52284169] [ 811.04443359] [ 512.79016113]
LeftLanesLength 24
RightLanesLength 16
y_max  720
y_min  480
vx_left, vy_left, cx_left, cy_left  [ 0.83076805] [-0.55661875] [ 434.9382019] [ 570.41497803]
vx_right , vy_right , cx_right , cy_right   [ 0.8549962] [ 0.51863432] [ 823.91668701] [ 517.99450684]
LeftLanesLength 32
RightLanesLength 10
y_max  720
y_min  480
vx_left, vy_left, cx_left, cy_left  [ 0.80902296] [-0.58777708] [ 444.68051147] [ 572.51031494]
vx_right , vy_right , cx_right , cy_right   [ 0.86682582] [ 0.49861103] [ 871.08929443] [ 545.85076904]
LeftLanesLength 36
RightLanesLength 14
y_max  720
y_min  480
vx_left, vy_left, cx_left, cy_left  [ 0.79990423] [-0.60012764] [ 473.05957031] [ 550.1050415]
vx_right , vy_right , cx_right , cy_right   [ 0.86539221] [ 0.5

...

LeftLanesLength 18
RightLanesLength 12
y_max  720
y_min  480
vx_left, vy_left, cx_left, cy_left  [ 0.78491908] [-0.61959833] [ 509.50897217] [ 531.90380859]
vx_right , vy_right , cx_right , cy_right   [ 0.86041903] [ 0.50958717] [ 854.98498535] [ 539.87438965]
LeftLanesLength 24
RightLanesLength 14
y_max  720
y_min  480
vx_left, vy_left, cx_left, cy_left  [ 0.77304655] [-0.63434929] [ 442.86328125] [ 583.10699463]
vx_right , vy_right , cx_right , cy_right   [ 0.84375906] [ 0.53672212] [ 827.03796387] [ 521.0512085]
LeftLanesLength 24
RightLanesLength 6
y_max  720
y_min  480
vx_left, vy_left, cx_left, cy_left  [ 0.80712676] [-0.59037817] [ 483.62393188] [ 545.60675049]
vx_right , vy_right , cx_right , cy_right   [ 0.85363847] [ 0.52086598] [ 880.13201904] [ 555.68029785]
LeftLanesLength 24
RightLanesLength 16
y_max  720
y_min  480
vx_left, vy_left, cx_left, cy_left  [ 0.78044605] [-0.6252231] [ 506.06399536] [ 530.20721436]
vx_right , vy_right , cx_right , cy_right   [ 0.85222965] [ 0.5

100%|█████████▉| 250/251 [00:17<00:00, 17.78it/s]

LeftLanesLength 32
RightLanesLength 24
y_max  720
y_min  480
vx_left, vy_left, cx_left, cy_left  [ 0.80570704] [-0.5923143] [ 513.32159424] [ 522.7701416]
vx_right , vy_right , cx_right , cy_right   [ 0.85023242] [ 0.52640748] [ 867.2802124] [ 545.97650146]
LeftLanesLength 42
RightLanesLength 18
y_max  720
y_min  480
vx_left, vy_left, cx_left, cy_left  [ 0.79482031] [-0.60684484] [ 486.34036255] [ 544.07513428]
vx_right , vy_right , cx_right , cy_right   [ 0.85582054] [ 0.51727283] [ 882.48052979] [ 555.9987793]
LeftLanesLength 30
RightLanesLength 20
y_max  720
y_min  480
vx_left, vy_left, cx_left, cy_left  [ 0.89061099] [-0.45476595] [ 370.91586304] [ 602.71716309]
vx_right , vy_right , cx_right , cy_right   [ 0.86905539] [ 0.49471483] [ 897.57104492] [ 559.6005249]
LeftLanesLength 28
RightLanesLength 28
y_max  720
y_min  480
vx_left, vy_left, cx_left, cy_left  [ 0.79050589] [-0.61245447] [ 439.05682373] [ 583.15673828]
vx_right , vy_right , cx_right , cy_right   [ 0.87012959] [ 0.492

100%|██████████| 251/251 [00:17<00:00, 14.18it/s]


LeftLanesLength 22
RightLanesLength 8
y_max  720
y_min  480
vx_left, vy_left, cx_left, cy_left  [ 0.79403919] [-0.60786659] [ 451.13366699] [ 574.0758667]
vx_right , vy_right , cx_right , cy_right   [ 0.88292605] [ 0.46951208] [ 796.75994873] [ 503.355896]
[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

CPU times: user 1min 38s, sys: 3.99 s, total: 1min 42s
Wall time: 18.6 s


In [12]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))